In [1]:
import pandas as pd

In [3]:
rollup = pd.read_excel("./data/CPUC PSPS Rollup.xlsx", header=1, parse_dates=True)

/Users/tomityrrell/opt/anaconda3/envs/TensorFlow/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [4]:
rollup.head()

,Utility,Outage Start,Full Restoration,Outage Duration,Outage Days,Outage Hours,Circuit Name,HFTD,TOTAL CUSTOMERS IMPACTED,RESIDENTIAL CUSTOMERS,COMMERCIAL/INDUSTRIAL CUSTOMERS,MEDICAL BASELINE CUSTOMERS,OTHER CUSTOMERS,Notes,Unnamed: 14,Unnamed: 15
0,SDG&E,2013-10-05 06:02:00,2013-10-05 14:55:00,"0 days,8 hrs,53 min",0.370139,8.883333,79,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SDG&E,2013-10-05 06:05:00,2013-10-05 14:41:00,"0 days,8 hrs,36 min",0.358333,8.600000,TL 626 - CIRCUIT 238,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SDG&E,2013-10-06 02:03:00,2013-10-06 08:10:00,"0 days,6 hrs,7 min",0.254861,6.116667,79,NaN,83.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SDG&E,2013-10-06 02:10:00,2013-10-06 08:10:00,"0 days,6 hrs,0 min",0.250000,6.000000,79,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SDG&E,2013-10-06 02:13:00,2013-10-06 07:44:00,"0 days,5 hrs,31 min",0.229861,5.516667,TL 626 - CIRCUIT 238,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
rollup.columns = rollup.columns.str.strip()
rollup.columns = rollup.columns.str.title()

In [6]:
dt_columns = ["Outage Start", "Full Restoration"]
for c in dt_columns:
    rollup[c] = pd.to_datetime(rollup[c], errors="coerce")

In [7]:
numeric_columns = ["Residential Customers", "Commercial/Industrial Customers", "Medical Baseline Customers", "Other Customers"]
for c in numeric_columns:
    rollup[c] = pd.to_numeric(rollup[c], errors='coerce')
    rollup[c].fillna(0, inplace=True)
    boolean_column = c.split(" ")[0].split("/")[0]
    rollup[boolean_column] = rollup[c] > 0

In [8]:
rollup.dtypes

Utility                                    object
Outage Start                       datetime64[ns]
Full Restoration                   datetime64[ns]
Outage Duration                            object
Outage Days                               float64
Outage Hours                              float64
Circuit Name                               object
Hftd                                       object
Total Customers Impacted                  float64
Residential Customers                     float64
Commercial/Industrial Customers           float64
Medical Baseline Customers                float64
Other Customers                           float64
Notes                                      object
Unnamed: 14                               float64
Unnamed: 15                               float64
Residential                                  bool
Commercial                                   bool
Medical                                      bool
Other                                        bool


In [9]:
sce = rollup[rollup.Utility == 'SCE'].copy()
sce.drop(["Notes", "Unnamed: 14", "Unnamed: 15"], axis=1, inplace=True)

In [10]:
sce.rename({"Circuit Name": "Circuit"}, axis=1, inplace=True)
sce["Circuit"] = sce["Circuit"].str.upper()
sce["Circuit"] = sce["Circuit"].str.strip()

In [11]:
sce = sce.sort_values("Outage Start").reset_index(drop=True)

In [13]:
sce.head()

,Utility,Outage Start,Full Restoration,Outage Duration,Outage Days,Outage Hours,Circuit,Hftd,Total Customers Impacted,Residential Customers,Commercial/Industrial Customers,Medical Baseline Customers,Other Customers,Residential,Commercial,Medical,Other
0,SCE,2019-01-01 02:27:00,2019-01-01 17:08:00,"0 days,14 hrs,41 min",0.611806,14.683333,TAIWAIN,XXX,34.0,0.0,0.0,0.0,0.0,False,False,False,False
1,SCE,2019-09-24 14:46:00,2019-09-24 20:34:00,"0 days,5 hrs,48 min",0.241667,5.800000,SADDLEBACK,TIER 3,9.0,9.0,0.0,0.0,0.0,True,False,False,False
2,SCE,2019-09-24 14:46:00,2019-09-24 20:34:00,"0 days,5 hrs,48 min",0.241667,5.800000,DYSART,TIER 3,76.0,76.0,0.0,0.0,0.0,True,False,False,False
3,SCE,2019-10-09 11:57:00,2019-10-09 19:41:00,"0 days,7 hrs,44 min",0.322222,7.733333,CANEBRAKE,Tier 2/Tier 3,63.0,63.0,0.0,0.0,0.0,True,False,False,False
4,SCE,2019-10-09 15:13:00,2019-10-09 19:27:00,"0 days,4 hrs,14 min",0.176389,4.233333,CANEBRAKE,Tier 2/Tier 3,122.0,101.0,0.0,0.0,0.0,True,False,False,False


In [14]:
sce.to_csv('./data/sce_rollup_2019.csv', index=False)